In [1]:
import requests
import json
import os
from nltk.corpus import stopwords
from nltk.stem import *

In [2]:
#Login inside FB: Go to URL : https://developers.facebook.com/
# Facebook for developers -> Tools and Support -> Graph API Explorer -> Access Token
#IMPORTANT: ACCESS_TOKEN expires very quickly(Probably, once every an hour or two)
access_token="EAACEdEose0cBANydY4bqq9Tss1jM9emyVy3FFCTrKlJhYIDIZCJ5DRokHHZCw1zmVcxjjfSZCZANpHgCQinifeWzpds8QVlrckpoK7kMqgypZAgHFnTiZAv2eVtt1SD0kYM2Y0qla3Y155beZBakAOFqVqpjlKus60ziNM7aQehxgZDZD"
#I have Found Out Some Group Names Manually , by browsing FB
groupNames=['Flats and Flatmates (Bangalore Chapter)','Flats Without Broker in India','Flats & Flatmates : Pan India!',
            'House for Rent in Bangalore','House/Flat for Rent in Bangalore without Brokers','Bangalore House Sharing, Flats, Rentals, Rooms for sharing available'
            ,'Flats without broker -bangalore','Flats and Flatmates : PAN Bangalore !','Flats / Flatmates without brokers in India' 
           ]
groupIDs=['1463274307229712','5561469939','340254516036729','219157614840863','450055131795193','355277761216963','314309115332224',
              '391928064244514','195366117186736']
#Only allowed to get POSTS posted on PUBLIC FB Groups: Out of these 9 groups, only 3 are Public:Hence have data for only 3 groups
# Another Issue: Currently. by Deafult API only provides latest 25 POSTS for each PUBLIC group
#Hence, Below method returnGroupFeeds() will only Fetch 75 posts for each execution
baseURL="https://graph.facebook.com/"
def returnGroupFeeds():
    respJSONDataList=[]
    for grpId in groupIDs:
        url=baseURL+grpId+"/feed?access_token="+access_token
        response=requests.get(url)
        if(response.status_code!=200):
            print(response)
            print("Please Update your Access-Token : It is Expired !!!")
            return
        respJSON=response.json()
        respJSONData=respJSON["data"]
        print(len(respJSONData)," Comments Found for GRP ID : ",grpId)
        respJSONDataList.extend(respJSONData)
    return respJSONDataList

In [3]:
jsonDataList=returnGroupFeeds()

0  Comments Found for GRP ID :  1463274307229712
0  Comments Found for GRP ID :  5561469939
25  Comments Found for GRP ID :  340254516036729
0  Comments Found for GRP ID :  219157614840863
25  Comments Found for GRP ID :  450055131795193
0  Comments Found for GRP ID :  355277761216963
0  Comments Found for GRP ID :  314309115332224
0  Comments Found for GRP ID :  391928064244514
24  Comments Found for GRP ID :  195366117186736


In [5]:
def extractPostsData(jsonDataList):
    posts=[{'from':jsonData.get('from'),'message':jsonData.get('message'),'containsPicture':'picture' in jsonData} for jsonData in jsonDataList]
    return posts

In [6]:
postsRelevantData=extractPostsData(jsonDataList)
#postsRelevantData=json.dumps(postsRelevantData)
postsRelevantData

[{'containsPicture': True,
  'from': {'id': '10152373624406396', 'name': 'Sushma Reddy'},
  'message': 'Our choices determine how happy or satisfied we are in our life. How do we make truly meaningful choices in life? Join us as we explore what it means to be an individual and how daring to choose can be the secret to attaining true freedom.\n\nLecture. Free Event.\n\nW: www.acropolis.org.in\nE: info@acropolis.org.in\nT: +91 22 22163712'},
 {'containsPicture': False,
  'from': {'id': '120573548325649', 'name': 'Ria Seth'},
  'message': 'room\n₹14 - Mumbai, India\n\n2 Room available for girls in Oshiwara \nFully furnished in very good building \nVery airy apartment with balcony \nRent -14.5 k each for sharing \nDeposit -30 k each \nWrite for more information . \nAvailable 7th December .'},
 {'containsPicture': False,
  'from': {'id': '10152534041647181', 'name': 'Sunayana Sahu'},
  'message': 'single occupancy room\n₹18,500 - Mumbai, India\n\nRequire a Female Flatmate for a fully furnis

In [8]:
#Uncomemnt Below 2 lines, to train the Model
#Ideally, Build an UI to enable Training(Training using Text Editor Sucks)
#with open('fbPostsTrainingData.model', 'w') as outfile:
#    json.dump(postsRelevantData, outfile)
postsTrainingData=[]
with open('fbPostsTrainingData.model') as data_file:    
    postsTrainingData = json.load(data_file)
print(len(postsTrainingData))

45


In [9]:
import nltk
from nltk.corpus import wordnet as wn
stemmer=PorterStemmer()

In [10]:
def getSynonymsUsingWordNet(words):
    synonymSet=set()
    for word in words:
        synonymSet.add(word)
        synonyms=wn.synsets(word)
        for synonym in synonyms:
            synName=synonym.name()
            synNameparts=synName.split(".")
            synName=synNameparts[0]
            synonymSet.add(synName)
    return synonymSet

In [11]:
#Features Related to Couting Named Entities within a POST : Didn't turnout to be a great feature :(
def extract_entity_names(entityTree):
    entity_names = []
    nltkNESet=('LOCATION','PERSON','ORGANIZATION','DATE','TIME','MONEY','PERCENT','FACILITY','GPE')
    if(hasattr(entityTree,'label') and (entityTree.label() in nltkNESet)):
        entity_names.append(entityTree)
    return entity_names
def findAllNEChunks(text):
    for sent in nltk.sent_tokenize(text):
        neChunks=nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)))
    return neChunks
def getNECountforAPost(postDesc):
    postDesc=postDesc.replace("\n",".")
    neChunks=findAllNEChunks(postDesc)
    entityNames = []
    for neChunk in neChunks:
        entityNames.extend(extract_entity_names(neChunk))
    return len(entityNames)

In [12]:
def getAmenitiesCount(postDesc):
    #List of Amenities - Courtesy : CommonFloor+FB Posts :-)
    #To avoid redundancy, I have STEMMED and LOWERCASED all of these amenities
    amenitiesSet=('garden','recreat','power','backup','clubhous','harvest','commun','swim',
                  'gym','gymnasium','indoor','intercom','tile','park','vitrifi','cupboard','kitchen',
                  'live','din','balconi','wardrob','granit','floor','wifi','tv','maid','servant',
                  'electr','badminton','tenni','basketball','basket','modular','microwav','groceri','store',
                  'oven','tataski','wash','aquaguard','play','fridg','attach','restroom','bathroom','separate',
                  'clubhous','intercom','gate','up','wardrob','cupboard','filter','hall','sofa','cupboard','tata'
                 'purifi','airport','busstand','busstop','bu','rack','room','tabl','furnitur','dine','televis',
                  'cctv','geyser','wi-fi','machin','secur','internet')
    postDesc=postDesc.lower()
    words=nltk.word_tokenize(postDesc)
    wordMembership=[stemmer.stem(word) in amenitiesSet for word in words]
    #requires Scaling ~~> Larger posts will have higher expected value of Amenities
        # To make the model robust to POST size ~~> Scale the count[Feature Scaling is important in many algorithms]
    return sum(wordMembership)/len(words)

In [13]:
#Used this module to get the  STEMS: ONE TIME EFFORT: SHOULDN'T RECOMPUTE
#tenantPhraseList=[('looking','apartment'),('looking','flat'),('looking','house'),('leads','please'),('any','leads')]
#tenantPhraseList.extend([('new','delhi'),('new','bangalore'),('new','chennai'),('new','pune'),('new','mumbai')])
#ownerPhraseList=[('looking','flatmate'),('looking','pg'),('looking','male'),('looking','female'),('looking','boy'),('looking','girl')]
#ownerPhraseList.extend([('available','apartment'),('available','flat'),('available','house')])
#ownerPhraseList.extend([('flatmate','required'),('need','flatmate'),('flat','mate')])
#ownerPhraseList.extend([('good','access'),('nice','locality'),('shared','accommodation'),('walking','distance')])
#stems=[stemmer.stem(w[1]) for w in ownerPhraseList]

In [14]:
def getStemmedBigrams(postDescription):
    postDescription=postDescription.replace("#"," ")
    commentWords=nltk.word_tokenize(postDescription)
    stopwordList=stopwords.words('english')
    stopwordList.extend([',',':','','(',')','&','#','.','!','!!','/','bhk','hk','rk','1bhk','1/2bhk','2bhk','2/3bhk','3bhk','3/4bhk','4bhk'])
    stopwordList.extend(range(9))
    content = [comment.lower() for comment in commentWords if comment.lower() not in stopwordList]
    stemmedContent=[stemmer.stem(cont) for cont in content]
    contentBigrams=list(nltk.bigrams(stemmedContent))
    return contentBigrams

In [15]:
#This method returns 1(Tenant/BUYER); -1(OWNER/SELLER); 0:Neutral [Categorical Feature]
def getAggregateOwnerTenantCat(postDescription):
    postDescription=postDescription.lower()
    tenantWordList=["budget"]
    ownersWordList=["flatmate","rent","deposit"]
    for tenantWord in tenantWordList:
        if postDescription.count(tenantWord)>0:
            return 1
    for ownersWord in ownersWordList:
        if postDescription.count(ownersWord)>0:
            return -1
    return 0

#Below method counts the frequency of matching Stemmed bigrams to Tenant Stemmed Bigrams
def getTenantPhraseCount(postDescription):
    #tenantPhraseListLev1 contains Stemmed Bigrams whose occurence exhibits very high 
        #probability of the POST posted by Tenant
    tenantPhraseListLev1=[('look','apart'),('look','flat'),('look','near'),('look','around'),('look','pg'),
                          ('look','hous'),('look','accommod'),('lead','pleas'),('ani','lead')]
    tenantPhraseList=[('seek','apart'),('seek','flat'),('seek','accommod'),('seek','hous'),('lead','pleas'),
                      ('ani','lead')]
    #Intuition is sentences like : "I am new to Bangalore and looking for ..."
    tenantPhraseList.extend([('new','delhi'),('new','bangalor'),('new','chennai'),('new','pune'),('new','mumbai')])
    count=0
    tenantBigrams=getStemmedBigrams(postDescription)
    for (w1,w2) in tenantBigrams:
        if(w1,w2) in tenantPhraseListLev1:
           count = count+3
        elif(w1,w2) in tenantPhraseList:
           count = count+1
    return count#No need to Scale:Length of post doesn't REALLY changes the fequency count

#Below method counts the frequency of matching Stemmed bigrams to Owner's Stemmed Bigrams
def getOwnerPhraseCount(postDescription) :
    ownerPhraseList=[('look','flatmat'),('look','male'),('look','femal'),('look','boy'),
                     ('look','girl'),('pg','girl'),('pg','boy'),('pg','male'),('pg','femal')]
    ownerPhraseList.extend([('avail','apart'),('avail','flat'),('avail','hous'),('avail','room'),('avail','sale'),
                            ('avail','rent')])
    ownerPhraseList.extend([('apart','avail'),('flat','avail'),('hous','avail')])
    #PG folks will say "Looking for a Shared Accomodation"
    ownerPhraseList.extend([('flatmat','requir'),('need','flatmat'),('need','roommat')
                            ,('flat','mate'),('share','accommod')])
    #Owners do tend to describe in terms of- "nice locality","good accessibility","walking distance from Landmark X"
    ownerPhraseList2=[('good','access'),('nice','local'),('walk','distanc')]
    count=0
    ownerBigrams=getStemmedBigrams(postDescription)
    for (w1,w2) in ownerBigrams:
        if(w1,w2) in ownerPhraseList:
           count=count+1
    for (w1,w2) in ownerBigrams:
        if(w1,w2) in ownerPhraseList2:
           count=count+1 #This Counting for the time being is not scaled: Needs Some Smart Scaling
    #No need to scale, Size doesn't really impact the frequency of these terms[ownerPhraseList2 has exception]
        #Ideally, there should be some scaling on ownerPharseList2 items occurence:Ignored for the time being
    return count
def doesContainPG(postDescription):
    postDescription=postDescription.replace("\n"," ")
    postDescription=postDescription.lower()
    listOfPGTerms=["pg","paying guest"]
    for pgterm in listOfPGTerms:
        if postDescription.find(pgterm) >= 0:
            return True
    return False
def generateFeatuesOnAFlatRelatedPost(postedData):
    postDescription=postedData['message']
    features = {}
    #features["count_NEs"]=getNECountforAPost(postDescription) #Seems to be a Bad Feature
    #features["NoOfPOstsMadebyUser"]=getNumOfPostsBySameUser(postData[0]) #Good feature, but don't have sufficient Data
    #features["description_size"]=len(postDescription) #Turned Out to be a Bad Feature with other Combinations
    features["containsImage"]=postedData['containsPicture']
    features["aggregatedOwnerTenantCat"]=getAggregateOwnerTenantCat(postDescription)
    features["count_tenant_Phrases"] = getTenantPhraseCount(postDescription)
    features["count_owner_Phrases"] = getOwnerPhraseCount(postDescription)
    features["count_Amenities"]=getAmenitiesCount(postDescription)
    features["isPGRelated"]=doesContainPG(postDescription)#Good feature;However Training Data might not contain PG, hence
        #the current Model at the moment mighn't be leveraging this feature
    return features

In [19]:
postTypeModel_train_set=[(generateFeatuesOnAFlatRelatedPost(postTrainData),postTrainData['typeOfPost']) for postTrainData in postsTrainingData]
len(postTypeModel_train_set)
train_set,test_set=postTypeModel_train_set[:30],postTypeModel_train_set[30:]#Trained on 30 posts, Will evaluate on 15 posts
#train_set,test_set=postTypeModel_train_set[:40],postTypeModel_train_set[5:]#Trained on 40 posts, Will evaluate on 5 posts:Since I have very less traing dataset and taht too is skewed
#train_set,test_set=postTypeModel_train_set[:44],postTypeModel_train_set[44:]#Trained on 44 posts, Will evaluate on 1 posts:Since I have very less traing dataset and taht too is skewed
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.9333333333333333


In [20]:
#ERROR ANALYSIS ON THE TRAINING DATASET
errors = []
for postTrainData in postsTrainingData:
    tag=postTrainData['typeOfPost']
    guess = classifier.classify(generateFeatuesOnAFlatRelatedPost(postTrainData))
    if guess != tag:
        errors.append( ("ACTUAL : "+tag, "GUESSED : "+guess, postTrainData) )
print(errors)

[('ACTUAL : SELL', 'GUESSED : BUY', {'from': {'id': '813074028765526', 'name': 'Sahil Duhan'}, 'typeOfPost': 'SELL', 'containsPicture': True, 'message': '\\ud83c\\udfea\\ud83c\\udfe1. \\u2618PG For Girls \\ud83c\\udf40 Sushant lok 1 , c block near park plaza hotel ,JUST 200 MTRS from huda city metro station GURGAON. WITHOUT BROKERAGE ACCOMODATION**\\ud83d\\udca6 fully furnished rooms\\ud83d\\udca6 \\ud83d\\udcbb32\\ L.E.D with tata sky \\ud83d\\udce1in all rooms \\ud83c\\udfa5 \\u2615Breakfast,lunch & dinner\\ud83c\\udf74 \\ud83c\\udf6a \\ud83d\\udebfAttach bathroom with geyser \\ud83d\\udec1 \\ud83d\\udd2624 Hr power & water back up\\ud83d\\udebf * fully secured (cctv camera + security guard 24*7) \\ud83c\\udf78Drinking r.o. water \\ud83d\\udeb0 \\ud83d\\udeaeDaily cleaning of rooms\\ud83d\\udeae \\ud83d\\udea1High Speed Wi-Fi Internet @ 8mbps \\ud83c\\udf74Quality food \\ud83c\\udfbdWashing Machine Available\\ud83c\\udfbd \\ud83c\\udf43Triple sharing @ 9k per bed \\ud83c\\udf40 \\ud8

In [21]:
#Run MODEL on DYNAMIC/NEW DATASETS
modelResults=[]
for newPost in postsRelevantData:
    if newPost is None or newPost['message'] is None:
        #print("<Data Issue>Will Skip this Post : ",newPost,"</Data Issue>")
        continue
    newPost['message']=newPost['message'].replace("\n"," ")
    #<IMPORTANT>SHOULD HAVE A SPELL CORRECTOR MODULE HERE..BEFORE RUNNING BAYESIAN MODEL</IMPORTANT>
    guess=classifier.classify(generateFeatuesOnAFlatRelatedPost(newPost))
    modelResults.append((guess,newPost['message']))
print(modelResults)

[('SELL', 'Our choices determine how happy or satisfied we are in our life. How do we make truly meaningful choices in life? Join us as we explore what it means to be an individual and how daring to choose can be the secret to attaining true freedom.  Lecture. Free Event.  W: www.acropolis.org.in E: info@acropolis.org.in T: +91 22 22163712'), ('SELL', 'room ₹14 - Mumbai, India  2 Room available for girls in Oshiwara  Fully furnished in very good building  Very airy apartment with balcony  Rent -14.5 k each for sharing  Deposit -30 k each  Write for more information .  Available 7th December .'), ('SELL', 'single occupancy room ₹18,500 - Mumbai, India  Require a Female Flatmate for a fully furnished 3BHK as a replacement from 1st January 2017 #Singleoccupancy #homeawayfromhome Rent - ₹ 18,500/- Brokerage - ₹ 0/- Deposit - ₹ 50,000/- Ideal for ppl working in BKC / Sion / Kurla / Santacruz (E) / Kalina - Fully furnished flat - Single Occupancy - Female only (Preferably Vegetarian &/ Egget